In [ ]:
import matplotlib.pyplot as plt          
import pandas as pd
import numpy as np                      
from scipy.sparse import csr_matrix      
%matplotlib inline

# Load in the dataset

In [ ]:
wiki = pd.read_csv('people_wiki.csv')
wiki.head(3)

# Extract word count vectors

For your convenience, we extracted the word count vectors from the dataset. The vectors are packaged in a sparse matrix, where the i-th row gives the word count vectors for the i-th document. Each column corresponds to a unique word appearing in the dataset. 

To load in the word count vectors, define the function

In [ ]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']   
    return csr_matrix( (data, indices, indptr), shape)

word_count = load_sparse_csr('people_wiki_word_count.npz')

# Find nearest neighbors using word count vectors

Let's start by finding the nearest neighbors of the Barack Obama page using the word count vectors to represent the articles and Euclidean distance to measure distance.  For this, we will use scikit-learn's implementation of k-nearest neighbors. We first create an instance of the NearestNeighbor class, specifying the model parameters. Then we call the fit() method to attach the training set.

In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(metric='euclidean', algorithm='brute')
model.fit(word_count)
print(wiki[wiki['name'] == 'Barack Obama'])

Let us run the k-nearest neighbor algorithm with Obama's article. Since the NearestNeighbor class expects a vector, we pass the 35817th row of word_count vector. The query returns the indices of and distances to the 10 nearest neighbors. To display the indices and distances together with the article name, run

In [ ]:
distances, indices = model.kneighbors(word_count[35817], n_neighbors=10) 
neighbors = pd.DataFrame(data={'distance':distances.flatten()}, index=indices.flatten())
wiki.join(neighbors).sort_values(['distance'])[['name', 'distance']]

# Interpreting the nearest neighbors

All of the 10 people are politicians, but about half of them have rather tenuous connections with Obama, other than the fact that they are politicians.

    Francisco Barrio is a Mexican politician, and a former governor of Chihuahua.
    Walter Mondale and Don Bonker are Democrats who made their career in late 1970s.
    Wynn Normington Hugh-Jones is a former British diplomat and Liberal Party official.
    Andy Anstett is a former politician in Manitoba, Canada.

Nearest neighbors with raw word counts got some things right, showing all politicians in the query result, but missed finer and important details.

For instance, let's find out why Francisco Barrio was considered a close neighbor of Obama. To do this, let's look at the most frequently used words in each of Barack Obama and Francisco Barrio's pages.

First, run the following cell to obtain the word_count column, which represents the word count vectors in the dictionary form. This way, we can quickly recognize words of great importance.

In [ ]:
def unpack_dict(matrix, map_index_to_word):
    table = sorted(map_index_to_word, key=map_index_to_word.get)
    data = matrix.data
    indices = matrix.indices
    indptr = matrix.indptr
    num_doc = matrix.shape[0]
    
    return [{k:v for k,v in zip([table[word_id] for word_id in indices[indptr[i]:indptr[i+1]] ],
                                 data[indptr[i]:indptr[i+1]].tolist())} \
               for i in range(num_doc) ]

In [ ]:
import json 
with open('people_wiki_map_index_to_word.json') as f:
    map_index_to_word = json.load(f)
wiki['word_count'] = unpack_dict(word_count, map_index_to_word)
wiki.head(2)

To make things even easier, we provide a utility function that displays a dictionary in tabular form:

In [ ]:
def top_words(name):
    """
    Get a table of the most frequent words in the given person's wikipedia page.
    """
    word_count = wiki[wiki['name'] == name]['word_count'].iloc[0]
    word_count = pd.DataFrame(word_count.items(), columns = ['word', 'count'])
    word_count = word_count.sort_values(['count'], ascending = False) 
    return word_count

obama_words = top_words('Barack Obama')
print(obama_words)
barrio_words = top_words('Francisco Barrio')
print(barrio_words)

Let's extract the list of most frequent words that appear in both Obama's and Barrio's documents. We've so far sorted all words from Obama and Barrio's articles by their word frequencies. We will now use a dataframe operation known as join. The join operation is very useful when it comes to playing around with data: it lets you combine the content of two tables using a shared column (in this case, the word column). See the documentation for more details.

For instance, running

In [ ]:
combined_words = pd.merge(obama_words, barrio_words, how='inner', on = 'word', suffixes=('_obama', '_barrio'))
combined_words.head(5)

Among the words that appear in both Barack Obama and Francisco Barrio, take the 5 that appear most frequently in Obama. How many of the articles in the Wikipedia dataset contain all of those 5 words?

In [ ]:
top_n_words = 5
common_words = set(obama_words.sort_values('count', ascending=False)['word'].iloc[0:top_n_words])

def has_top_words(word_count_vector):
    # extract the keys of word_count_vector and convert it to a set
    unique_words = set(word_count_vector.keys())   # YOUR CODE HERE
    # return True if common_words is a subset of unique_words
    # return False otherwise
    return common_words.issubset(unique_words) 

wiki['has_top_words'] = wiki['word_count'].apply(has_top_words)

# use has_top_words column to answer the quiz question
len(wiki[wiki['has_top_words'] == True]) # YOUR CODE HERE

Checkpoint. Check your has_top_words function on two random articles: 

In [ ]:
print('Output from your function:', has_top_words(wiki.iloc[32]['word_count']))
print('Correct output: True')
print('Also check the length of unique_words. It should be 167')
print('Output from your function:', has_top_words(wiki.iloc[33]['word_count']))
print('Correct output: False')
print('Also check the length of unique_words. It should be 188')

In [ ]:
obama_page = wiki[wiki['name'] == 'Barack Obama'].index[0]
obama_page

Measure the pairwise distance between the Wikipedia pages of Barack Obama, George W. Bush, and Joe Biden. Which of the three pairs has the smallest distance?

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

obama_page_index = wiki[wiki['name'] == 'Barack Obama'].index[0]
bush_page_index = wiki[wiki['name'] == 'George W. Bush'].index[0]
biden_page_index = wiki[wiki['name'] == 'Joe Biden'].index[0]

print('DISTANCE BETWEEN OBAMA AND BUSH', euclidean_distances(word_count[obama_page_index], Y=word_count[bush_page_index]))
print('DISTANCE BETWEEN OBAMA AND BIDEN', euclidean_distances(word_count[obama_page_index], Y=word_count[biden_page_index]))
print('DISTANCE BETWEEN BIDEN AND BUSH', euclidean_distances(word_count[biden_page_index], Y=word_count[bush_page_index]))

Collect all words that appear both in Barack Obama and George W. Bush pages. Out of those words, find the 10 words that show up most often in Obama's page.

In [ ]:
obama_words = top_words('Barack Obama')
bush_words = top_words('George W. Bush')
combined_words_obama_bush = pd.merge(obama_words, bush_words, how='inner', on = 'word', suffixes=('_obama', '_bush'))
combined_words_obama_bush.head(5)

# Extract the TF-IDF vectors

Much of the perceived commonalities between Obama and Barrio were due to occurrences of extremely frequent words, such as "the", "and", and "his". So the nearest neighbors algorithm is recommending plausible results sometimes for the wrong reasons.

To retrieve articles that are more relevant, we should focus more on rare words that don't happen in every article. TF-IDF (term frequency–inverse document frequency) is a feature representation that penalizes words that are too common. Let us load in the TF-IDF vectors and repeat the nearest neighbor search.

For your convenience, we extracted the TF-IDF vectors from the dataset. The vectors are packaged in a sparse matrix, where the i-th row gives the TF-IDF vectors for the i-th document. Each column corresponds to a unique word appearing in the dataset. The mapping between words and integer indices are given in people_wiki_map_index_to_word.gl.

To load in the TF-IDF vectors, run. In addition to the sparse matrix, we also store the TF-IDF vectors in dictionary form as well, to allow for easy interpretation.

In [ ]:
tf_idf = load_sparse_csr('people_wiki_tf_idf.npz')
wiki['tf_idf'] = unpack_dict(tf_idf, map_index_to_word)
wiki.head(2)

# Find nearest neighbors using TF-IDF vectors

Since we are now using a different set of features, we should create a new nearest neighbor model. Create another instance of the NearestNeighbor class as follows. Then call the fit() method to associate it with the TF-IDF vectors.

In [ ]:
model_tf_idf = NearestNeighbors(metric='euclidean', algorithm='brute')
model_tf_idf.fit(tf_idf)

Perform the nearest neighbor search by running

In [ ]:
distances, indices = model_tf_idf.kneighbors(tf_idf[35817], n_neighbors=10) 
neighbors = pd.DataFrame(data={'distance':distances.flatten()}, index=indices.flatten())
wiki.join(neighbors).sort_values(['distance'])[['name', 'distance']]

Let's determine whether this list makes sense.

    With a notable exception of Roland Grossenbacher, the other 8 are all American politicians who are contemporaries of Barack Obama.

    Phil Schiliro, Jesse Lee, Samantha Power, and Eric Stern worked for Obama.

Clearly, the results are more plausible with the use of TF-IDF. Let's take a look at the word vector for Obama and Schilirio's pages. Notice that TF-IDF representation assigns a weight to each word. This weight captures relative importance of that word in the document. Let us sort the words in Obama's article by their TF-IDF weights; we do the same for Schiliro's article as well.

In [ ]:
def top_words_tf_idf(name):
    word_count =  wiki[wiki['name'] == name]['tf_idf'].iloc[0]
    word_count = pd.DataFrame(word_count.items(), columns = ['word', 'weight'])
    return word_count.sort_values(['weight'], ascending=False)

obama_tf_idf = top_words_tf_idf('Barack Obama')
print(obama_tf_idf)

schiliro_tf_idf = top_words_tf_idf('Phil Schiliro')
print(schiliro_tf_idf)

 Using the join operation we learned earlier, try your hands at computing the common words shared by Obama's and Schiliro's articles. Sort the common words by their TF-IDF weights in Obama's document. The first 10 words should say: Obama, law, democratic, Senate, presidential, president, policy, states, office, 2011.

In [ ]:
obama_words = top_words_tf_idf('Barack Obama')
schiliro_words = top_words_tf_idf('Phil Schiliro')
combined_words_obama_schiliro = pd.merge(obama_words, schiliro_words, how='inner', on = 'word', suffixes=('_obama', '_schiliro'))
combined_words_obama_schiliro.head(5)

Among the words that appear in both Barack Obama and Phil Schiliro, take the 5 that have largest weights in Obama. How many of the articles in the Wikipedia dataset contain all of those 5 words?

In [ ]:
top_n_words = 5
common_words = set(combined_words_obama_schiliro.sort_values('weight_obama', ascending=False)['word'].iloc[0:top_n_words])
def has_top_words(word_count_vector):
    # extract the keys of word_count_vector and convert it to a set
    unique_words = set(word_count_vector.keys())   # YOUR CODE HERE
    # return True if common_words is a subset of unique_words
    # return False otherwise
    return common_words.issubset(unique_words)

wiki['has_top_words'] = wiki['word_count'].apply(has_top_words)

# use has_top_words column to answer the quiz question
len(wiki[wiki['has_top_words'] == True])

 Notice the huge difference in this calculation using TF-IDF scores instead of raw word counts. We've eliminated noise arising from extremely common words.

# Choosing metrics

you may wonder why Joe Biden, Obama's running mate in two presidential elections, is missing from the query results of model_tf_idf. Let's find out why. First, compute the distance between TF-IDF features of Obama and Biden.

The distance is larger than the distances we found for the 10 nearest neighbors. But one may wonder, is Biden's article that different from Obama's, more so than, say, Schiliro's? It turns out that, when we compute nearest neighbors using the Euclidean distances, we unwittingly favor short articles over long ones. Let us compute the length of each Wikipedia document, and examine the document lengths for the 100 nearest neighbors to Obama's page.

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

obama_page_index = wiki[wiki['name'] == 'Barack Obama'].index[0]
biden_page_index = wiki[wiki['name'] == 'Joe Biden'].index[0]

print('DISTANCE BETWEEN OBAMA AND BIDEN', euclidean_distances(tf_idf[obama_page_index], Y=tf_idf[bush_page_index]))

In [ ]:
# Comptue length of all documents
def compute_length(row):
    return len(row.split(' '))
wiki['length'] = wiki['text'].apply(compute_length)

# Compute 100 nearest neighbors and display their lengths
distances, indices = model_tf_idf.kneighbors(tf_idf[35817], n_neighbors=100)
neighbors = pd.DataFrame(data = {'distance':distances.flatten()}, index = indices.flatten())
nearest_neighbors_euclidean = wiki.join(neighbors).sort_values(['distance'])[['name', 'length', 'distance']]
nearest_neighbors_euclidean.head(10)

To remove this bias, we turn to cosine distances. Cosine distances let us compare word distributions of two articles of varying lengths.

Let us train a new nearest neighbor model, this time with cosine distances. We then repeat the search for Obama's 100 nearest neighbors.

In [ ]:
model2_tf_idf = NearestNeighbors(algorithm='brute', metric='cosine')
model2_tf_idf.fit(tf_idf)
distances, indices = model2_tf_idf.kneighbors(tf_idf[35817], n_neighbors=100)
neighbors = pd.DataFrame(data = {'distance':distances.flatten()}, index = indices.flatten())
nearest_neighbors_euclidean = wiki.join(neighbors).sort_values(['distance'])[['name', 'length', 'distance']]
nearest_neighbors_euclidean.head(10)

# Problem with cosine distances: tweets vs. long articles

Happily ever after? Not so fast. Cosine distances ignore all document lengths, which may be great in certain situations but not in others. For instance, consider the following (admittedly contrived) example.

In [ ]:
+--------------------------------------------------------+
|                                             +--------+ |
|  One that shall not be named                | Follow | |
|  @username                                  +--------+ |
|                                                        |
|  Democratic governments control law in response to     |
|  popular act.                                          |
|                                                        |
|  8:05 AM - 16 May 2016                                 |
|                                                        |
|  Reply   Retweet (1,332)   Like (300)                  |
|                                                        |
+--------------------------------------------------------+

How similar is this tweet to Barack Obama's Wikipedia article? Let's transform the tweet into TF-IDF features, using an encoder fit to the Wikipedia dataset. (That is, let's treat this tweet as an article in our Wikipedia dataset and see what happens.)

In [ ]:
tweet = {'act': 3.4597778278724887,
 'control': 3.721765211295327,
 'democratic': 3.1026721743330414,
 'governments': 4.167571323949673,
 'in': 0.0009654063501214492,
 'law': 2.4538226269605703,
 'popular': 2.764478952022998,
 'response': 4.261461747058352,
 'to': 0.04694493768179923}

Let's look at the TF-IDF vectors for this tweet and for Barack Obama's Wikipedia entry, just to visually see their differences.

In [ ]:
word_indices = []
for word in tweet.keys():
    if word in map_index_to_word.keys():
        word_indices.append(map_index_to_word[word]  )

tweet_tf_idf = csr_matrix( (list(tweet.values()), ([0]*len(word_indices), word_indices)),
                          shape=(1, tf_idf.shape[1]) )

now, compute the cosine distance between the Barack Obama article and this tweet:

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

obama_tf_idf = tf_idf[35817]
print(cosine_distances(obama_tf_idf, tweet_tf_idf))

et's compare this distance to the distance between the Barack Obama article and all of its Wikipedia 10 nearest neighbors:

In [ ]:
distances, indices = model2_tf_idf.kneighbors(obama_tf_idf, n_neighbors=10)
print(distances)

With cosine distances, the tweet is "nearer" to Barack Obama than everyone else, except for Joe Biden! This probably is not something we want. If someone is reading the Barack Obama Wikipedia page, would you want to recommend they read this tweet? Ignoring article lengths completely resulted in nonsensical results. In practice, it is common to enforce maximum or minimum document lengths. After all, when someone is reading a long article from The Atlantic, you wouldn't recommend him/her a tweet.